In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 速度运动模型

速度运动模型（Velocity Motion Model）认为可以通过两个速度：一个旋转的和一个平移的速度，来控制机器人。驱动系统通常是通过这种方式控制的，包括差分驱动、阿克曼驱动和同步驱动。*不包括没有非完整约束的驱动系统，如万向轮（Mecanum wheels）的机器人或有腿机器人。*

用$\upsilon_t$表示$t$时刻的平移速度（Translation Velocity），用$\omega_t$表示旋转速度（Rotation Velocity）。因此，有：
$$u_t = \begin{bmatrix} \upsilon_t \\ \omega_t \end{bmatrix}$$
并规定，逆时针旋转为（向左转），$\omega_t$为正；向前运动，$\upsilon_t$为正。

## 1. 精确的运动

![](./img/velocity.jpg)
> 从$(x \quad y \quad \theta)^T$开始，并以恒定速度$\upsilon$和$\omega$运动的无噪声机器人完成的运动。

令$u_t = (\upsilon \quad \omega)^T$表示时刻$t$的控制，如果两种速度在整个时间间隔内$[t, t-1]$都是固定的值，则机器人以如下半径的圆运动：
$$r = \left| \frac {\upsilon}{\omega} \right|$$

这是任意半径为$r$圆形轨迹运动的对象在平移速度$\upsilon$和旋转速度$\omega$之间都遵循的一般关系：
$$\upsilon = \omega r$$

**注意：**机器人根本不旋转（$\omega = 0$）的情况，在这种情况下机器人沿直线运动，此时$r$为无穷大。

令$x_{t-1} = (x \, y \, \theta)^T$是机器人的初始位姿，并假定在$\Delta t$时间内保持速度$(\upsilon \, \omega)^T$恒定。很容易看出，圆的中心位于：
$$x_c = x - \frac {\upsilon}{\omega}sin \theta$$

$$y_c = y + \frac {\upsilon}{\omega}cos \theta$$

变量$(x_c \, y_c)^T$表示坐标。运动$\Delta t$时间后，理想机器人将位于$x_t = (x' \, y' \, \theta')^T$且：

$$\begin{bmatrix} x' \\ y\ \\ \theta' \end{bmatrix} = 
\begin{bmatrix} 
x_c + \frac {\upsilon}{\omega} sin(\theta + \omega \Delta t) \\ 
y_c - \frac {\upsilon}{\omega} cos(\theta + \omega \Delta t) \\
\theta + \omega \Delta t
\end{bmatrix} = \begin{bmatrix} x \\ y \\ \theta \end{bmatrix} + 
\begin{bmatrix}
- \frac {\upsilon}{\omega} sin \theta + \frac {\upsilon}{\omega} sin(\theta + \omega \Delta t) \\ 
\frac {\upsilon}{\omega} cos \theta - \frac {\upsilon}{\omega} cos(\theta + \omega \Delta t) \\
\omega \Delta t
\end{bmatrix}
$$

尽管真实的机器人不能从一个速度跳到下一个速度，也不能保证在每一个时间间隔内速度保持恒定，利用很小的时间间隔$\Delta t$，在每一个时间间隔用一个常值近似真实的速度，是很常用的方法。

## 2. 真实运动

